# **Starbucks Offer Model**

**Background Information**

The dataset you will be provided in this portfolio exercise was originally used as a take-home assignment provided by Starbucks for their job candidates. The data for this exercise consists of about 120,000 data points split in a 2:1 ratio among training and test files. In the experiment simulated by the data, an advertising promotion was tested to see if it would bring more customers to purchase a specific product priced at $10.  
Since it costs the company 0.15 to send out each promotion, it would be best to limit that promotion only to those that are most receptive to the promotion. Each data point includes one column indicating whether or not an individual was sent a promotion for the product, and one column indicating whether or not that individual eventually purchased that product. Each individual also has seven additional features associated with them, which are provided abstractly as V1-V7.

**Optimization Strategy**

Your task is to use the training data to understand what patterns in V1-V7 indicate that a promotion should be provided to a user. Specifically, your goal is to maximize the following metrics:
* **Incremental Response Rate (IRR)**

IRR depicts how many more customers purchased the product with the promotion, as compared to if they didn't receive the promotion. Mathematically, it's the ratio of the number of purchasers in the promotion group to the total number of customers in the purchasers group (treatment) minus the ratio of the number of purchasers in the non-promotional group to the total number of customers in the non-promotional group (control).

$$ IRR = \frac{purch_{treat}}{cust_{treat}} - \frac{purch_{ctrl}}{cust_{ctrl}} $$


* **Net Incremental Revenue (NIR)**

NIR depicts how much is made (or lost) by sending out the promotion. Mathematically, this is 10 times the total number of purchasers that received the promotion minus 0.15 times the number of promotions sent out, minus 10 times the number of purchasers who were not given the promotion.

$$ NIR=(10 * purch_{treat} − 0.15 * cust_{treat}) − 10* purch_{ctrl} $$

**How To Test Your Strategy**  
When you feel like you have an optimization strategy, complete the `promotion_strategy` function to pass to the `test_results` function.  
Table of actual promotion vs. predicted promotion customers: 

|           | Actual |  |
|-----------|-----|----|
| **Predicted** | Yes | No |
| Yes |  I | II |
| No  | III| IV |

The metrics are only being compared for the individuals we predict should obtain the promotion – that is, quadrants I and II. Since the first set of individuals that receive the promotion (in the training set) receive it randomly, we can expect that quadrants I and II will have approximately equivalent participants.  
Comparing quadrant I to II then gives an idea of how well your promotion strategy will work in the future. 
See how each variable or combination of variables along with a promotion influences the chance of purchasing. When you feel like you have a strategy for who should receive a promotion, test your strategy against the test dataset used in the final `test_results` function.

In [1]:
# load in packages
from itertools import combinations

from test_results import test_results, score
import numpy as np
import pandas as pd
import scipy as sp

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, f1_score, confusion_matrix,\
roc_curve, roc_auc_score, precision_score, recall_score, precision_recall_curve

import matplotlib.pyplot as plt
import seaborn as sb
%matplotlib inline

In [2]:
train_data = pd.read_csv('./data/training.csv')
train_data.head()

,ID,Promotion,purchase,V1,V2,V3,V4,V5,V6,V7
0,1,No,0,2,30.443518,-1.165083,1,1,3,2
1,3,No,0,3,32.159350,-0.645617,2,3,2,2
2,4,No,0,2,30.431659,0.133583,1,1,4,2
3,5,No,0,0,26.588914,-0.212728,2,1,4,2
4,8,Yes,0,3,28.044332,-0.385883,1,1,2,2


In [3]:
train_data.shape

(84534, 10)

In [4]:
table = train_data.groupby(['Promotion', 'purchase']).size()
table = table.reset_index()
table.rename({0: 'count'}, axis=1, inplace=True)
table.pivot_table(index=['Promotion'], columns=['purchase'], values='count')

purchase,0,1
Promotion,,
No,41851,319
Yes,41643,721


In [6]:
train_data.iloc[:, 3:]

,V1,V2,V3,V4,V5,V6,V7
0,2,30.443518,-1.165083,1,1,3,2
1,3,32.159350,-0.645617,2,3,2,2
2,2,30.431659,0.133583,1,1,4,2
3,0,26.588914,-0.212728,2,1,4,2
4,3,28.044332,-0.385883,1,1,2,2
...,...,...,...,...,...,...,...
84529,1,30.084876,1.345672,1,1,3,1
84530,3,33.501485,-0.299306,1,1,4,1
84531,1,31.492019,1.085939,2,3,2,2
84532,1,37.766106,0.999361,2,2,1,2


In [40]:
print('Baseline Model Performance')
print('---')

baseline_preds = train_data['Promotion'].apply(lambda x: 1 if x == 'Yes' else 0)
y_actual = train_data['purchase']

# performance
print(f'Accuracy Score: {accuracy_score(y_actual, baseline_preds)}')
print(f'Confusion Matrix: \n{confusion_matrix(y_actual, baseline_preds)}')
print(f'Area Under Curve: {roc_auc_score(y_actual, baseline_preds)}')
print(f'Recall Score: {recall_score(y_actual, baseline_preds)}')
print(f'F1 Score: {f1_score(y_actual, baseline_preds)}')

Baseline Model Performance
---
Accuracy Score: 0.5036080157096553
Confusion Matrix: 
[[41851 41643]
 [  319   721]]
Area Under Curve: 0.5972574146276748
Recall Score: 0.6932692307692307
F1 Score: 0.033222744447516356


In [21]:
feature_cols = ['V1', 'V2', 'V3', 'V4', 'V5', 'V6', 'V7']
X = train_data[feature_cols]
y = train_data['purchase']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
clf = LogisticRegression(random_state=42)

clf.fit(X_train, y_train)
clf.score(X_test, y_test)

print('Model 1 Performance')
print('---')

# test
y_pred = clf.predict(X_test)

# performance
print(f'Accuracy Score: {accuracy_score(y_test,y_pred)}')
print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall Score: {recall_score(y_test,y_pred)}')
print(f'F1 Score: {f1_score(y_test, y_pred)}')

Model 1 Performance
---
Accuracy Score: 0.9876974034423611
Confusion Matrix: 
[[16699     0]
 [  208     0]]
Area Under Curve: 0.5
Recall Score: 0.0
F1 Score: 0.0


In [42]:
train_data.groupby(['purchase']).size()

purchase
0    83494
1     1040
dtype: int64

In [46]:
# define class weights
w = {0:1, 1:83}

clf_w = LogisticRegression(class_weight=w, random_state=42)

clf_w.fit(X_train, y_train)

print('Model 2 Performance')
print('---')

# test
y_pred = clf_w.predict(X_test)

# performance
print(f'Accuracy Score: {accuracy_score(y_test,y_pred)}')
print(f'Confusion Matrix: \n{confusion_matrix(y_test, y_pred)}')
print(f'Area Under Curve: {roc_auc_score(y_test, y_pred)}')
print(f'Recall Score: {recall_score(y_test,y_pred)}')
print(f'F1 Score: {f1_score(y_test, y_pred)}')

Model 2 Performance
---
Accuracy Score: 0.3896019400248418
Confusion Matrix: 
[[ 6443 10256]
 [   64   144]]
Area Under Curve: 0.5390695896115383
Recall Score: 0.6923076923076923
F1 Score: 0.027149321266968326


In [5]:
def promotion_strategy(df):
    '''
    INPUT 
    df - a dataframe with *only* the columns V1 - V7 (same as train_data)

    OUTPUT
    promotion_df - np.array with the values
                   'Yes' or 'No' related to whether or not an 
                   individual should recieve a promotion 
                   should be the length of df.shape[0]
                
    Ex:
    INPUT: df
    
    V1  V2    V3  V4  V5  V6  V7
    2   30  -1.1   1   1   3   2
    3   32  -0.6   2   3   2   2
    2   30  0.13   1   1   4   2
    
    OUTPUT: promotion
    
    array(['Yes', 'Yes', 'No'])
    indicating the first two users would recieve the promotion and 
    the last should not.
    '''
    
    
    
    return promotion